# Mise en valeur des caractéristiques musicales qui influent sur la popularité d'une musique

Voici les deux bases de données sur lesquelles on a pu travailler :
- Popularité, genre et caractéristiques : (https://www.kaggle.com/zaheenhamidani/ultimate-spotify-tracks-db)
- Régions, nombre d'écoutes et évolution de la popularité : (https://www.kaggle.com/dhruvildave/spotify-charts)

Les fichiers qui y sont liés sont dans le dossier resources et sont nommés SpotifyFeatures.csv, charts.csv.

In [87]:
import plotly_express as px
import pandas as pd
import plotly as pt
import numpy as np
import plotly.graph_objects as go

import dash
from dash import dcc
from dash import html

pd.options.plotting.backend = "plotly"

## Découpage des bases de données

La base de données initialement récuperée sur kaggle avait une taille de 3GB de données, cela ne rendant pas pertinant l'utilisation de l'entièreté de cette dernière. En effet, cette base de données est mise à jour depuis début 2017 jusq'à fin 2021 où chaque jour des nouvelles données sont ajoutées.

Les données de décembre n'étant pas à jour, nous avons choisi celles de novembre qui sont donc les données complètes les plus récentes.

Les commandes pour trim la base de données se trouvent dans le fichier "data/get_data.py".

Il vous faut l'exécuter depuis la racine du projet si vous voulez utiliser ce notebook.


## Merging

Début de la database **SpotifyFeatures.csv**

In [90]:
df = pd.read_csv("data/SpotifyFeatures.csv")

df.head()

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,99373,0.910,0.000,C#,0.3460,-1.828,Major,0.0525,166.969,4/4,0.814
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.246,0.590,137373,0.737,0.000,F#,0.1510,-5.559,Minor,0.0868,174.003,4/4,0.816
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.952,0.663,170267,0.131,0.000,C,0.1030,-13.879,Minor,0.0362,99.488,5/4,0.368
3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.703,0.240,152427,0.326,0.000,C#,0.0985,-12.178,Major,0.0395,171.758,4/4,0.227
4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.950,0.331,82625,0.225,0.123,F,0.2020,-21.150,Major,0.0456,140.576,4/4,0.390


Début de la database **charts.csv**

In [91]:
df2 = pd.read_csv("data/charts.csv")

df2.head()

,Unnamed: 0,title,rank,date,artist,url,region,chart,trend,streams
0,0,Bar,1,2021-11-30,"TINI, L-Gante",https://open.spotify.com/track/0lJE8f0lx8mUSfM...,Argentina,top200,SAME_POSITION,491835.0
1,1,Salimo de Noche,2,2021-11-30,"Tiago PZK, Trueno",https://open.spotify.com/track/6P096SaOzrgGxs8...,Argentina,top200,SAME_POSITION,379567.0
2,2,DANCE CRIP,3,2021-11-30,Trueno,https://open.spotify.com/track/1kjs5eR4DMzyB0g...,Argentina,top200,SAME_POSITION,366315.0
3,3,Una Vaina Loca,4,2021-11-30,"Fuego, Manuel Turizo, Duki",https://open.spotify.com/track/2qEoAz0i6yEz5dP...,Argentina,top200,SAME_POSITION,269979.0
4,4,Antes de Ti,5,2021-11-30,"Rusherking, Maria Becerra",https://open.spotify.com/track/23R68uWGdp07SwQ...,Argentina,top200,SAME_POSITION,249624.0


In [92]:
df2 = df2.drop(columns=['Unnamed: 0'])

In [93]:
df = df.rename(columns={'artist_name':'artist', 'track_name':'title'})

Ici on va vouloir créer un tableau qui fait la moyenne des caracteristiques sur toutes les musiques (environ 10000 par genre) pour chaque genre.

D'abord on enlever les colonnes qui ne correspondent pas aux caractéristiques et au genre,
puis on regroupe les genres pour avoir une moyenne

In [94]:
dfmean = df.drop(columns=['artist', 'title', 'track_id', 'key', 'mode', 'time_signature'])

In [95]:
dfmean_grouped = dfmean.groupby(['genre']).mean()

## Affichage

### Diagramme Bubble

Ce diagramme a pour but de mettre en valeure les caractéristiques musicales des musiques spotify

On peut l'adapter en changeant l'axe x

In [96]:
fig = px.scatter(dfmean_grouped, x='danceability', y='popularity', color=dfmean_grouped.index, hover_name=dfmean_grouped.index, log_x=False,
                    title="Popularité des genres à travers la caractéristique dancabilité"
                )
fig

### Diagramme Bar

Ce diagramme va avoir pour but d'observer le nombres d'écoutes par musiques dentre le top200 et le top 50 d'un genre en fonction de son pays d'émission à une date précise.

Pour cela on aura besoin de joindre les 2 datasets pour avoir acces aux nombres d'écoutes et aux pays d'émisssion

* url2trackid(url) aura pour but de recuperer l'identifiant d'une musique spotify dans un url pour ensuite la rejoindre avec l'autre tableau

In [97]:
def url2trackid(url):
    splited_url = url.split("/")
    return splited_url[len(splited_url) - 1]

Ici on va appliquer la fonction sur tous le dataset charts.csv

In [98]:
clean_df2 = df2
clean_df2['url'] = clean_df2['url'].apply(url2trackid)
clean_df2 = clean_df2.rename(columns={'url':'track_id'})
clean_df2

,title,rank,date,artist,track_id,region,chart,trend,streams
0,Bar,1,2021-11-30,"TINI, L-Gante",0lJE8f0lx8mUSfMyxeYpiC,Argentina,top200,SAME_POSITION,491835.0
1,Salimo de Noche,2,2021-11-30,"Tiago PZK, Trueno",6P096SaOzrgGxs8NWOfX7M,Argentina,top200,SAME_POSITION,379567.0
2,DANCE CRIP,3,2021-11-30,Trueno,1kjs5eR4DMzyB0gm1Z8waV,Argentina,top200,SAME_POSITION,366315.0
3,Una Vaina Loca,4,2021-11-30,"Fuego, Manuel Turizo, Duki",2qEoAz0i6yEz5dPggABcLH,Argentina,top200,SAME_POSITION,269979.0
4,Antes de Ti,5,2021-11-30,"Rusherking, Maria Becerra",23R68uWGdp07SwQdK8mWzI,Argentina,top200,SAME_POSITION,249624.0
...,...,...,...,...,...,...,...,...,...
16815,Our Love (From the series Arcane League of Leg...,46,2021-11-30,"Curtis Harding, Jazmine Sullivan",4OiGC4aSeZV2O71xKl6ssk,Vietnam,viral50,MOVE_DOWN,NaN
16816,Nuestra Canción,47,2021-11-30,"Monsieur Periné, Vicente Garcia",5reQI13tWWYDLMrGcUF4Mk,Vietnam,viral50,MOVE_DOWN,NaN
16817,Bananza (Belly Dancer),48,2021-11-30,Akon,0dNPpDFdvzj6XFIRzg2hmp,Vietnam,viral50,NEW_ENTRY,NaN
16818,ONLY,49,2021-11-30,LeeHi,6TBJkXHPhu3EsMk1bshwuI,Vietnam,viral50,MOVE_UP,NaN


Ici on va joindre les tableaus tout en gardant les données utile au graphe

In [99]:
join_df = pd.merge(df, clean_df2, how ='inner', on =['track_id', 'artist', 'title'])
join_df = join_df.drop(columns=['artist', 'title', 'track_id', 'popularity', 'acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'speechiness', 'tempo', 'time_signature', 'valence', 'rank', 'date',	'chart', 'trend', 'key', 'liveness', 'mode', 'loudness'])
join_df = join_df.where(join_df['region'] != 'Global')

In [100]:
join_df['region'].nunique()

69

In [101]:
join_df['genre'].unique()

array(['Alternative', "Children's Music", 'Pop', 'R&B', nan, 'Rock',
       'Hip-Hop', 'Dance', 'Indie', 'Folk', 'Ska', 'Rap', 'Soul', 'Anime',
       'Blues', 'Reggaeton', 'Reggae', 'Jazz', 'Comedy', 'Movie',
       'Country'], dtype=object)

Pour le graphe on va checker le nombre de pays d'émission pour identifier si genre a beaucoup d'écoute pour ensuite afficher les musiques écoutées plus de 10000 ou 1000000 à la date du graphe

In [102]:
def genre_bar(genre):
    idf_curr = join_df.where(join_df['genre'] == genre)
    if idf_curr['region'].where(idf_curr['streams'] > 10000.0).nunique() < 10:
        idf_curr = idf_curr.where(idf_curr['streams'] > 10000.0).dropna()
    else:
        idf_curr = idf_curr.where(idf_curr['streams'] > 100000.0).dropna()
    fig2 = px.bar(idf_curr, x="region", y="streams", color_discrete_sequence=['purple'],
        title = ("Nombres d'écoutes par musiques dentre le top200 et le top 50 du genre \"{0}\" en fonction de son pays d'émission à la date du 30 novembre 2021".format(genre)))
    return fig2

In [103]:
genre_bar("Pop")

#### Les problèmes liés au diagramme bar

Ce diagramme reste difficile à mettre en avant car les musiques étant catégorisées viral50 n'ont pas de données pour le nombres d'écoutes donc le graphe ne représente que les musiques du top200 au top50.

De plus, les pays identifiés sur le graphe sont les pays d'émission; ce qui est nettement moins intéressant que les pays d'écoute.

### Diagramme Polaire

Ce diagramme a pour but de résumer les caractéristiques d'un genre, ceci est intéressant pour comparer avec les autres genres visuellement parlant.

On a aussi remarqué que la caractéristique speechiness n'était pas pertinente car elle varie entre 0 et 0.2 sauf pour la Comedy. Donc si on enleve le genre Comedy dans le premier diagramme, on a une meilleure vu de la carctéristique.

In [104]:
def iris(genre):
  genre_df = dfmean.where(dfmean['genre'] == genre).dropna().groupby(['genre']).mean()
  
  features=[genre_df['popularity'],genre_df['acousticness'],genre_df['danceability'],genre_df['energy'],genre_df['instrumentalness'],genre_df['speechiness'],
        genre_df['valence'],genre_df['liveness'],genre_df['loudness']]
  ranges=[100.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -60.0]

  for index in range(len(ranges)):
    features[index] = round(features[index]/ranges[index], 3)

  list = []

  for i in range(len(features)):
    print(features[i].values[0])
    list.append(features[i].values[0])

  fig = go.Figure(data=go.Scatterpolar(
    r = list,
    theta = ['popularity', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'speechiness', 'valence', 'liveness', 'loudness'],
    fill = 'toself'
    ))

  fig.update_layout(
    polar=dict(
      radialaxis=dict(
        visible=True,range=[0,1.0]
      ),
    ),
    showlegend=False,
    title = 'Diagramme polaire des caractéristiques du genre "' + str(genre) + '"',
    title_x=0.5
  )

  return fig

In [105]:
iris('Movie').show()

0.122
0.697
0.491
0.371
0.125
0.109
0.448
0.229
0.216
